In [1]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CICIDS2017_df, CTU13_df = pd.concat(CICIDS2017_df_list, ignore_index=True), pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [3]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [4]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [5]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [6]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [7]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max',
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std',
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size',
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
])

In [8]:
# Train a Binary Dummy Classifier on CTU13 dataset
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])

dummy_bin_ctu13 = DummyClassifier(strategy='stratified')
dummy_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
    
# Test on CTU13 dataset
predictions_bin_ctu13 = dummy_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Acc (CTU13 Binary Dummy): 0.517121
F1-score (CTU13 Binary Dummy): 0.429445


Pred,Benign,Malicious
True,,
Benign,12371,8955
Malicious,8856,6703


In [9]:
# Train a Binary Dummy Classifier on CICIDS2017 dataset
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

dummy_bin_cicids = DummyClassifier(strategy='stratified')
dummy_bin_cicids.fit(train_cicids[features], train_cicids['GT'])

# Test on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Binary Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary Dummy): 0.684064
F1-score (CICIDS2017 Binary Dummy): 0.197797


Pred,Benign,Malicious
True,,
Benign,729722,178806
Malicious,178565,44058


In [10]:
# Train a Multiclass Dummy Classifier on CICIDS2017 dataset
dummy_multi_cicids = DummyClassifier(strategy='stratified')
dummy_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])

# Test on CICIDS2017 dataset
predictions_multi_cicids = dummy_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass Dummy): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("F1-score (CICIDS2017 Multiclass Dummy): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='macro')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multiclass Dummy): 0.657845
F1-score (CICIDS2017 Multiclass Dummy): 0.066762


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,730659,647,40868,3236,73645,1777,1835,2509,4,12,50835,1811,480,12,198
Bot,641,1,41,1,60,0,1,2,0,0,35,0,0,0,0
DDoS,41205,38,2324,178,4134,131,109,127,0,0,2854,116,33,1,11
DoS GoldenEye,3407,2,208,21,313,9,7,11,0,0,221,5,3,0,0
DoS Hulk,73757,69,4238,324,7475,185,192,242,0,3,5092,199,61,0,21
DoS Slowhttptest,1761,1,103,15,183,4,1,5,0,0,129,3,2,0,0
DoS slowloris,1867,2,105,10,208,1,4,4,0,0,139,1,1,0,1
FTP-Patator,2621,6,155,8,270,8,6,9,0,0,159,7,3,0,2
Heartbleed,4,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [11]:
# Test the binary Dummy Classifier trained on CTU13 dataset on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_ctu13.predict(test_cicids[features])
print("Acc (CTU13 to CICIDS2017 Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CTU13 to CICIDS2017 Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CTU13 to CICIDS2017 Dummy): 0.547131
F1-score (CTU13 to CICIDS2017 Dummy): 0.268475


Pred,Benign,Malicious
True,,
Benign,524886,383642
Malicious,128621,94002


In [12]:
# Test the binary Dummy Classifier trained on CICIDS2017 on CTU13 dataset
predictions_bin = dummy_bin_cicids.predict(test_ctu13[features])
print("Acc (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin)))
print("F1-score (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary to CTU13 Dummy): 0.545669
F1-score (CICIDS2017 Binary to CTU13 Dummy): 0.264871


Pred,Benign,Malicious
True,,
Benign,17108,4218
Malicious,12540,3019


In [13]:
# Test the multiclass Dummy Classifier trained on CICIDS2017 on CTU13 dataset
predictions_multi = dummy_multi_cicids.predict(test_ctu13[features])
print("Acc (CICIDS2017 Multi-class to CTU13 Dummy): {:3f}".format(accuracy_score(test_ctu13[' Label'], predictions_multi)))
print("F1-score (CICIDS2017 Multi-class to CTU13 Dummy): {:3f}".format(f1_score(test_ctu13[' Label'], predictions_multi, average='macro')))
pd.crosstab(test_ctu13[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multi-class to CTU13 Dummy): 0.465880
F1-score (CICIDS2017 Multi-class to CTU13 Dummy): 0.051951


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,
BENIGN,17176,14,934,71,1692,39,38,58,1245,39,14,0,6
Bot,12440,8,710,53,1297,30,31,45,909,25,8,1,2
